In [1]:
import requests
url = "http://localhost:8000"
private_url = "http://localhost:8800"
create_user = f"{private_url}/user/"
token_url = f"{url}/token"
vehicle = f"{url}/twin/vehicle/"
charger = f"{url}/twin/charge-point/"
user="test@emobility.com"
password="Test1234"


In [2]:
def get_token(user, password):
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    
    data = {
        'grant_type': '',
        'username': user,
        'password': password,
        'scope': '',
        'client_id': '',
        'client_secret': '',
    }
    
    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code == 200:
        print("token generated")
        token = response.json().get("access_token")
    else:
        print(response.text)
    
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json',
    }
    return headers

In [3]:
headers = get_token(user=user, password=password)

Internal Server Error


UnboundLocalError: cannot access local variable 'token' where it is not associated with a value

In [ ]:
headers

In [ ]:
def create_vehicles(name='Renault e-Dokker', 
                    battery_capacity_kwh=100, maximum_charging_rate_kw=50, number_of_vehicles=10): 
    vehicles = []
    for i in range( number_of_vehicles):
        json_data = {
            'name': f'{name} {i}',
            'battery_capacity': battery_capacity_kwh,
            'maximum_charging_rate': maximum_charging_rate_kw
        }
        
        response = requests.post(vehicle, headers=headers, json=json_data)
        if response.status_code == 200:
            print("Vechicle created")
            vehicle_output = response.json()
            vehicles.append(vehicle_output)
        else:
            print(response.text)
    return vehicles

### Create vehicles

In [ ]:
vehicles = create_vehicles(name='Renault e-Dokker', 
                    battery_capacity_kwh=100, maximum_charging_rate_kw=50, number_of_vehicles=1)

### Create chargepoints
- each chargepoint has two evses with one connector each. You can modify this to your liking.

In [ ]:
def create_charge_points(name, max_dc_power_kw, max_ac_power_kw, csms_url,
                        connector_type, number_chargers):
    charge_points = []
    for i in range(number_chargers):
        json_data = {
              "name": f"{name} {i}",
              "maximum_dc_power": max_dc_power_kw,
              "maximum_ac_power": max_ac_power_kw,
              "csms_url": csms_url,
              "evses": [
                  {"connectors":[{"connector_type": connector_type}]}, 
                  {"connectors":[{"connector_type": connector_type}]}]
            }
            
        response = requests.post(charger, headers=headers, json=json_data)
            
        if response.status_code == 200:
            print("Charge point created")
            charger_output = response.json()
            charge_points.append(charger_output)
        else:
            print(response.text)
    return charge_points

In [ ]:
charge_points = create_charge_points(name='Marseille charger', max_dc_power_kw=150,
                                     max_ac_power_kw=20, 
                                     csms_url="ws://csms:9000",
                                     connector_type="cCCS1", number_chargers=1)

### Connect the chargers so they are available for charging

In [ ]:
def connect_chargers(charge_points):
    for charger in charge_points:
        json_data = {
                "charge_point_id": charger.get("id")
            }
            
        response = requests.post(url=url + "/twin/charge-point/action/connect-charger", headers=headers, json=json_data)
            
        if response.status_code == 200:
            print(response.json())
        else:
            print(response.text)

In [ ]:
connect_chargers(charge_points=charge_points)

### Start charging sessions
- In the example below we start the charging session on the first connector of the first evse

In [ ]:
from time import sleep

def start_transactions(charge_points, vehicles, number_of_start_transactions=10):
    transactions = []
    for charger, vehicle in zip(charge_points[:number_of_start_transactions],
                                vehicles[:number_of_start_transactions]):
        sleep(1)
    
        json_data = {
            "connector_id": charger.get("evses")[0].get("connectors")[0].get("id"),
            "vehicle_id": vehicle.get("id")
        }
        
        connect = url+"/twin/charge-point/action/start-transaction"
        response = requests.post(connect, headers=headers, json=json_data)
        
        if response.status_code == 200:
            transaction = response.json()
            transactions.append(transaction)
            print("done")
        else:
            print(response.text)
    return transactions


In [ ]:
transactions = start_transactions(charge_points=charge_points, vehicles=vehicles, number_of_start_transactions=10)

### Stop transactions
- Using the transaction id we stop each transaction

In [ ]:
def stop_transactions(transactions):
    for transaction in transactions:
        json_data = {
            "transaction_id": transaction.get("id")
    }
        
        connect = url + "/twin/charge-point/action/stop-transaction"
        response = requests.post(connect, headers=headers, json=json_data)
        
        if response.status_code == 200:
            print(response.json())
        else:
            print(response.text)

In [ ]:
stop_transactions(transactions=transactions)